In [1]:
from Bio.SeqIO.FastaIO import SimpleFastaParser
import os, sys
import pandas as pd

In [2]:
os.getcwd()

'/wynton/home/ahituv/fongsl/nullomers/bin-lock/sei'

In [3]:
BED2FA="/wynton/home/ahituv/fongsl/tools/genome/fasta_from_bed.py"
BED = './gapdh.bed'

In [7]:
def extendBed(bed, flanksize):
    """
    expand bed
    
    require
        bedtools slop
        wynton
    
    input
        bed (str) - path to bed file
        flanksize (int) - length to extend bed coordinates by
    
    method 
        1. get genome size. 
        2. make outfile name
        3. bedtools slop command
        
    return
        out (str) - path to results.bed
    """
    genome = "/wynton/home/ahituv/fongsl/dna/hg38/hg38.chrom.sizes"
    
    path, file= os.path.split(bed)
    out = os.path.join(path, file.strip(".bed") + f".{flanksize}.bed")
    cmd = f"bedtools slop -i {bed} -g {genome} -b {flanksize} > {out}"
    os.system(cmd)
    
    return out

In [8]:
# Assumes the bed file annotates 1 single nucleotide
flanksize = 2048
out = extendBed(BED, flanksize)

Error: The requested file (./gapdh.bed) could not be opened. Error message: (No such file or directory). Exiting!


# bed -> FA

In [21]:
cmd = f"python {BED2FA} {os.path.join(os.getcwd(),out)} -b hg38"
os.system(cmd)

fa = os.path.join(os.getcwd(),out).strip(".bed") + ".fa"
print(fa)

wrote /wynton/home/ahituv/fongsl/nullomers/bin-lock/./gapdh.2048.fa
/wynton/home/ahituv/fongsl/nullomers/bin-lock/./gapdh.2048.fa


# inserts

In [52]:
def makeInsertDict(file):
    
    insert_dict = {}
    val=0
    with open(file, "r") as reader:
        for line in reader:
            insert_dict[val] = line.strip("\n")
            val+=1
        reader.close()
    return insert_dict

In [53]:
ENDOG_14mers="/wynton/home/ahituv/fongsl/dna/hs1/kmers/14mers/14mer.firstorder.pam.purine.nohomopoly.GC.txt"

insert_dict = makeInsertDict(ENDOG_14mers)

In [57]:
insert_seq = insert_dict[0]

# insert nullomer into 4096 window

- rn, only test first 100

In [60]:
def seqInsert(seq, insert_seq):

    insert_size = len(insert_seq)
    
    # find center of sequence, subtract 1/2 of insert length, get center start
    insert_start = len(seq)/2 - (insert_size/2)
    insert_end = insert_start + insert_size  # find center end
    

    return seq[:int(insert_start)] + insert_seq + seq[int(insert_end):]

In [85]:
INSERT_SIZE = 23

outfasta = fa.strip('.fa') + ".100.14mers.fa"
fasta_writer = open(outfasta, "w")

# parse the fasta
with open(fa, "r") as fasta_reader:
    
    for values in SimpleFastaParser(fasta_reader):

        name, seq = values
        
        # only write the first 100 sequences
        for n in np.arange(100):
            
            insert_seq = insert_dict[n] # get insert

            new_seq = seqInsert(seq, insert_seq)  # get new seq w insert at center 

            newname = f">{insert_seq}"  # write new name
            
            #print(newname, "\n", new_seq, len(new_seq), len(seq))
            
            # write to fasta
            row=f"{newname}\n{new_seq}\n"
            fasta_writer.write(row)

    fasta_writer.close()

In [86]:
row

'>ATGTGTTGCTACGCGATCGACGG\nTAGGAAGTCTGTTCCTGTCCTCCCTGTGCAGGGTATCCTGTAGGGTGACCTGGAATTCGAATTCTGTTTCCCTTGTAAAATATTTGTCTGTCTCTTTTTTTTAAAAAAAAAAAAggccgggcactgtggctcacgcctgtaatcccagcactttgcgataccaaggcgggtggataacctgaggtagggagttcgagaccagcctgaccaacatggagaaaccccatctctactaaaaataaaaaattagccgggcgtattggcgtgcgcctgtaatcccagctactcaagaggctgaggcaggagaatcgcctgaacccagaggcggaggttgtagtgagccgaaatcacaccattgcactccagcttgggcaacaatagcgaacctccatctcaaattaaaaaaaaaaTGCCTACACGCTCTTTAAAATGCAAGGCTTTCTCTTAAATTAGCCTAACTGAACTGCGTTGAGCTGCTTCAACTTTGGAATATATGTTTGCCAATCTCCTTGTTTTCTAATGAATAAATGTTTTTATATACTTTTAGACATTTTTTCCTAAGCTTGTCTTTGTTTCATCTTTCACATTAGCCCAGTTTCATGCAGCAGAGAGAGGGTTATCAGTGCAGAGAGAGATGAGTGAGCCCAGAGTCCTAGGGCCTGTCCCGGGATGGCAGATGAGCTTCCTGCCCCGTCACTGCCACCTTTCCCCTCTCAACCTCTGGACCCTGCACAGTGACCAGACAGCCTCTCTGGGGAGAATTATGCAGTGCCTAGGCTCCAGATcagtgcttctgaaccgggggcaattttgtctgccagaggacatctgacaacacctggggcctgttttgttgtcatagcctataggggaagaatgctaccagcatttgtgggaagaggccagggatgtggctcaacatcctgcagtgcacaggatggcccctcaacaaagaatcacacggcccacaatg

# launch sei

In [82]:
BUILD = "hg38"
OUTDIR = os.getcwd()
SCRIPT = "/wynton/home/ahituv/fongsl/bin/sei-framework/sarah_scripts/launch_qsub.py"
cmd = ["python",
       SCRIPT,
      outfasta,
      BUILD, 
      OUTDIR
      ]
print(" ".join(cmd))
#os.system(" ".join(cmd))

sh /wynton/home/ahituv/fongsl/bin/sei-framework/1_sequence_prediction.sh /wynton/home/ahituv/fongsl/nullomers/bin-lock/./gapdh.2048.100.14mers.fa hg38 /wynton/home/ahituv/fongsl/nullomers/bin-lock/gapdh_pred
Input argments: /wynton/home/ahituv/fongsl/nullomers/bin-lock/./gapdh.2048.100.14mers.fa /wynton/home/ahituv/fongsl/nullomers/bin-lock/gapdh_pred hg38 
use_cuda: False
0.2600111961364746


Traceback (most recent call last):
  File "/wynton/home/ahituv/fongsl/bin/sei-framework/1_sequence_prediction.py", line 23, in <module>
    from docopt import docopt
ModuleNotFoundError: No module named 'docopt'
